Imports

In [2]:
import sqlite3
import datetime
import pandas as pd
import math
import numpy as np
import requests
import json

Connection to db file

In [4]:
connnection = sqlite3.connect("sqlite.db")
cursor = connnection.cursor()

Initial tests

In [5]:
asd = cursor.execute("SELECT Data FROM Czas")
asd.fetchone()

('2026-02-17',)

In [28]:
date_to_raw = datetime.datetime.now().date()

Methods

In [73]:
def finance_year(month, year):
    if month > 6:
        return year+1
    return year

In [74]:
def quarter_year(month):
    return math.ceil(month/3)

In [142]:
def adding_missing_dates(x):
    missing_dates = pd.DataFrame(x)
    missing_dates.rename(columns={ 'Czas_Data' : 'Data'}, inplace=True)
    temp_kwartal = []
    temp_rok_finansowy = []

    for i in range(len(missing_dates)):
        temp_kwartal.append(quarter_year(missing_dates["Data"][i].month))
        temp_rok_finansowy.append(finance_year(missing_dates["Data"][i].month, missing_dates["Data"][i].year))

    missing_dates["Kwartal"] = temp_kwartal
    missing_dates["Rok_finansowy"] = temp_rok_finansowy
    try:
        missing_dates.to_sql('Czas', connnection, if_exists='append', index=False)
    except:
        print("Date already exists")

Table Czas initialization 

In [37]:
time_raws =[]
for i in range(1000):
    time_raws.append([date_to_raw, quarter_year(date_to_raw.month), finance_year(date_to_raw.month,date_to_raw.year)])
    date_to_raw += datetime.timedelta(days=1)

In [38]:
cursor.executemany("insert into Czas values(?,?,?)", time_raws)

In [41]:
connnection.commit()

Table Geografia initialization

In [44]:
df = pd.read_csv("economist_country_codes.csv", sep=";")

In [65]:
date_start =[]
for i in range(df["COUNTRY|CODE"].size):
    date_start.append(datetime.datetime.now().date())
df['Start_Data'] = date_start
df['End_Data'] = pd.NaT

In [70]:
df.rename(columns={'COUNTRY|CODE': 'Kraj'}, inplace=True)

In [71]:
df.to_sql('Geografia', connnection, if_exists='append', index=False)

58

Table BMI initialization - since the dates in dataset are earlier than initialy initialized dates in table Czas, there'll be a need to add those specific dates later

In [7]:
bmi = pd.read_sql("select Kraj from Geografia", connnection)

In [15]:
country_code = []

for raw in bmi['Kraj']:
    country_code.append(str(raw).split('|'))
#country_code[i][1]

'AUS'

API datapoints

In [112]:
urls_local_price = []
urls_dollar_price = []
urls_dollar_ppp = []

for i in range(len(country_code)):
    urls_local_price.append(f"https://data.nasdaq.com/api/v3/datasets/ECONOMIST/BIGMAC_{country_code[i][1]}?column_index=1&api_key=oCZQmK4DSs6pkpCsQurZ")
    urls_dollar_price.append(f"https://data.nasdaq.com/api/v3/datasets/ECONOMIST/BIGMAC_{country_code[i][1]}?column_index=3&api_key=oCZQmK4DSs6pkpCsQurZ")
    urls_dollar_ppp.append(f"https://data.nasdaq.com/api/v3/datasets/ECONOMIST/BIGMAC_{country_code[i][1]}?column_index=4&api_key=oCZQmK4DSs6pkpCsQurZ")
    

In [143]:
for i, raw in enumerate(country_code):
    local_price_json = requests.get(urls_local_price[i]).json()
    dollar_price_json = requests.get(urls_dollar_price[i]).json()
    dollar_ppp_json = requests.get(urls_dollar_ppp[i]).json()
    
    bmi_full = pd.DataFrame(local_price_json['dataset']['data'], columns=["Czas_Data","Local_Price"])
    bmi_full['Czas_Data'] = pd.to_datetime(bmi_full['Czas_Data'])
    
    adding_missing_dates(bmi_full['Czas_Data'])

    temp = pd.DataFrame(dollar_price_json['dataset']['data'], columns=["to_drop","Dollar_Price"])
    bmi_full["Dollar_Price"] = temp['Dollar_Price']
    temp = pd.DataFrame(dollar_ppp_json['dataset']['data'], columns=["to_drop","Dolar_PPP"])
    bmi_full["Dolar_PPP"] = temp['Dolar_PPP']
    bmi_full["Geografia_ID_Kraj"] = country_code[i][0] + "|" + country_code[i][1]

    bmi_full.to_sql('BigMacIndex', connnection, if_exists='append', index=False)
    bmi_full = pd.DataFrame(columns=bmi_full.columns)

Date already exists
Date already exists
Date already exists
Date already exists
Date already exists
Date already exists
Date already exists
Date already exists
Date already exists
Date already exists
Date already exists
Date already exists
Date already exists
Date already exists
Date already exists
Date already exists
Date already exists
Date already exists
Date already exists
Date already exists
Date already exists
Date already exists
Date already exists
Date already exists
Date already exists
Date already exists
Date already exists
Date already exists
Date already exists
Date already exists
Date already exists
Date already exists
Date already exists
Date already exists
Date already exists
Date already exists
Date already exists
Date already exists
Date already exists
Date already exists
Date already exists
Date already exists
Date already exists
Date already exists
Date already exists
Date already exists
Date already exists
Date already exists
Date already exists
Date already exists


In [144]:
connnection.close()